In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [ ]:
fiqa = pd.read_csv('/content/gdrive/MyDrive/fiqa_data.csv')
forex = pd.read_csv('/content/gdrive/MyDrive/sentiment_annotated_with_texts.csv')
test = pd.read_csv('/content/gdrive/MyDrive/test_1106.csv')

In [ ]:
forex = forex[['text','true_sentiment']].copy()
forex.rename(columns={'true_sentiment': 'label'}, inplace=True)
forex['label'] = forex['label'].str.lower()
forex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2291 entries, 0 to 2290
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2291 non-null   object
 1   label   2291 non-null   object
dtypes: object(2)
memory usage: 35.9+ KB


In [ ]:
fiqa = fiqa[['text','Label1']].copy()
fiqa.rename(columns={'Label1': 'label'}, inplace=True)

In [ ]:
print(fiqa.info())
print(forex.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1113 entries, 0 to 1112
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1113 non-null   object
 1   label   1113 non-null   object
dtypes: object(2)
memory usage: 17.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2291 entries, 0 to 2290
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2291 non-null   object
 1   label   2291 non-null   object
dtypes: object(2)
memory usage: 35.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970 entries, 0 to 969
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    970 non-null    object
 1   label   970 non-null    object
dtypes: object(2)
memory usage: 15.3+ KB
None


In [ ]:
!pip install transformers[torch] accelerate -U
!pip install datasets
import torch
import random
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from datasets import Dataset, load_metric
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import numpy as np
from torch.cuda.amp import GradScaler, autocast
from torch.autograd import Function
import torch.optim as optim

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Define the label list
label_list = ['positive', 'negative', 'neutral']
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}

# Load the different models
FinBERT_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", num_labels=len(label_list))
TinyBERT_model = AutoModelForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=len(label_list))
Augmented_FinBERT_model = AutoModelForSequenceClassification.from_pretrained('/content/gdrive/MyDrive/finetuned_finbert_25052024_v6_GPT4o_rephrased_and_GPT_DFT_STL', num_labels=len(label_list))
TinyFinBERT_model = AutoModelForSequenceClassification.from_pretrained('/content/gdrive/MyDrive/finetuned_tinyfinbert_12062024_v2', num_labels=len(label_list))

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", do_lower_case=True)

# Load datasets
test_dataset = Dataset.from_pandas(test)
forex_dataset = Dataset.from_pandas(forex)
fiqa_dataset = Dataset.from_pandas(fiqa)

# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=64)

test_dataset = test_dataset.map(tokenize_function, batched=True)
forex_dataset = forex_dataset.map(tokenize_function, batched=True)
fiqa_dataset = fiqa_dataset.map(tokenize_function, batched=True)

# Encode the labels
def encode_labels(examples):
    examples['label'] = [label_to_id[label] for label in examples['label']]
    return examples

test_dataset = test_dataset.map(encode_labels, batched=True)
forex_dataset = forex_dataset.map(encode_labels, batched=True)
fiqa_dataset = fiqa_dataset.map(encode_labels, batched=True)

# Set the format for PyTorch
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
forex_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
fiqa_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create DataLoaders
test_loader = DataLoader(test_dataset, batch_size=32)
forex_loader = DataLoader(forex_dataset, batch_size=32)
fiqa_loader = DataLoader(fiqa_dataset, batch_size=32)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/970 [00:00<?, ? examples/s]

Map:   0%|          | 0/2291 [00:00<?, ? examples/s]

Map:   0%|          | 0/1113 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

Map:   0%|          | 0/2291 [00:00<?, ? examples/s]

Map:   0%|          | 0/1113 [00:00<?, ? examples/s]

In [ ]:
def evaluate(model, eval_loader, device,label_col='label'):
    model.eval()
    accuracy_metric = load_metric('accuracy', trust_remote_code=True)
    f1_metric = load_metric('f1', trust_remote_code=True)
    precision_metric = load_metric('precision', trust_remote_code=True)
    recall_metric = load_metric('recall', trust_remote_code=True)

    for batch in eval_loader:
        inputs = {key: val.to(device) for key, val in batch.items() if key in tokenizer.model_input_names}
        labels = batch[label_col].to(device)

        with torch.no_grad():
            outputs = model(**inputs)
        predictions = outputs.logits.argmax(dim=-1)

        accuracy_metric.add_batch(predictions=predictions, references=labels)
        f1_metric.add_batch(predictions=predictions, references=labels)
        precision_metric.add_batch(predictions=predictions, references=labels)
        recall_metric.add_batch(predictions=predictions, references=labels)

    results = {
    'accuracy': accuracy_metric.compute()['accuracy'],
    'f1': f1_metric.compute(average='weighted')['f1'],
    'precision': precision_metric.compute(average='weighted')['precision'],
    'recall': recall_metric.compute(average='weighted')['recall']
    }
    return results

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
FinBERT_model = FinBERT_model.to(device)

In [ ]:
TinyBERT_model = TinyBERT_model.to(device)

In [ ]:
Augmented_FinBERT_model = Augmented_FinBERT_model.to(device)

In [ ]:
TinyFinBERT_model = TinyFinBERT_model.to(device)

In [ ]:
def print_metrics(dataset_name, metrics):
    print(f"Metrics for {dataset_name}:")
    print(f"Accuracy: {metrics['accuracy']}")
    print(f"F1 Score: {metrics['f1']}")
    print(f"Precision: {metrics['precision']}")
    print(f"Recall: {metrics['recall']}")
    print("------")

In [ ]:
# Assuming fiqa_loader is a PyTorch DataLoader
for batch in fiqa_loader:
    print("Batch keys:", batch.keys())  # This prints the dictionary keys if batch is a dict
    # Optionally, print shapes and data types of the batch content
    for key, value in batch.items():
        print(f"{key}: Type {type(value)}, Shape {value.shape if hasattr(value, 'shape') else 'N/A'}")
    break  # Only process the first batch to see the structure


Batch keys: dict_keys(['label', 'input_ids', 'attention_mask'])
label: Type <class 'torch.Tensor'>, Shape torch.Size([32])
input_ids: Type <class 'torch.Tensor'>, Shape torch.Size([32, 64])
attention_mask: Type <class 'torch.Tensor'>, Shape torch.Size([32, 64])


In [ ]:
# Evaluate FinBERT model on the test data
FinBERT_test_metrics = evaluate(FinBERT_model, test_loader, device)
print_metrics("FinBERT Test", FinBERT_test_metrics)

FinBERT_forex_metrics = evaluate(FinBERT_model, forex_loader, device)
print_metrics("FinBERT Forex", FinBERT_forex_metrics)

FinBERT_fiqa_metrics1 = evaluate(FinBERT_model, fiqa_loader, device)
print_metrics("FinBERT FIQA", FinBERT_fiqa_metrics1)

Metrics for FinBERT Test:
Accuracy: 0.8422680412371134
F1 Score: 0.8438634016026945
Precision: 0.8544671272734454
Recall: 0.8422680412371134
------
Metrics for FinBERT Forex:
Accuracy: 0.48013967699694454
F1 Score: 0.4448524360727107
Precision: 0.49876417852212535
Recall: 0.48013967699694454
------
Metrics for FinBERT FIQA:
Accuracy: 0.5265049415992812
F1 Score: 0.556318177171707
Precision: 0.6641863042333894
Recall: 0.5265049415992812
------


In [ ]:
# Evaluate TinyBERT model on the test data
TinyBERT_test_metrics = evaluate(TinyBERT_model, test_loader, device)
print_metrics("TinyBERT Test", TinyBERT_test_metrics)

TinyBERT_forex_metrics = evaluate(TinyBERT_model, forex_loader, device)
print_metrics("TinyBERT Forex", TinyBERT_forex_metrics)

TinyBERT_fiqa_metrics1 = evaluate(TinyBERT_model, fiqa_loader, device)
print_metrics("TinyBERT FIQA Label1", TinyBERT_fiqa_metrics1)

Metrics for TinyBERT Test:
Accuracy: 0.13298969072164948
F1 Score: 0.03290890920930817
Precision: 0.6102506423309417
Recall: 0.13298969072164948
------


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for TinyBERT Forex:
Accuracy: 0.30947184635530334
F1 Score: 0.14627702604394005
Precision: 0.09577282368656048
Recall: 0.30947184635530334
------
Metrics for TinyBERT FIQA Label1:
Accuracy: 0.09254267744833783
F1 Score: 0.015703287936044145
Precision: 0.008579564155876504
Recall: 0.09254267744833783
------


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Evaluate Augmented_FinBERT model on the test data
Augmented_FinBERT_test_metrics = evaluate(Augmented_FinBERT_model, test_loader, device)
print_metrics("Augmented_FinBERT Test", Augmented_FinBERT_test_metrics)

Augmented_FinBERT_forex_metrics = evaluate(Augmented_FinBERT_model, forex_loader, device)
print_metrics("Augmented_FinBERT Forex", Augmented_FinBERT_forex_metrics)

Augmented_FinBERT_fiqa_metrics1 = evaluate(Augmented_FinBERT_model, fiqa_loader, device)
print_metrics("Augmented_FinBERT FIQA", Augmented_FinBERT_fiqa_metrics1)

Metrics for Augmented_FinBERT Test:
Accuracy: 0.8742268041237113
F1 Score: 0.8738641064373833
Precision: 0.8742802923563459
Recall: 0.8742268041237113
------
Metrics for Augmented_FinBERT Forex:
Accuracy: 0.49498035792230466
F1 Score: 0.47971371422861786
Precision: 0.5080937115306186
Recall: 0.49498035792230466
------
Metrics for Augmented_FinBERT FIQA:
Accuracy: 0.6217430368373764
F1 Score: 0.6384510674282899
Precision: 0.6709575946689273
Recall: 0.6217430368373764
------


In [ ]:
# Evaluate TinyFinBERT model on the test data
TinyFinBERT_test_metrics = evaluate(TinyFinBERT_model, test_loader, device)
print_metrics("TinyFinBERT Test", TinyFinBERT_test_metrics)

TinyFinBERT_forex_metrics = evaluate(TinyFinBERT_model, forex_loader, device)
print_metrics("TinyFinBERT Forex", TinyFinBERT_forex_metrics)

TinyFinBERT_fiqa_metrics1 = evaluate(TinyFinBERT_model, fiqa_loader, device)
print_metrics("TinyFinBERT FIQA", TinyFinBERT_fiqa_metrics1)

Metrics for TinyFinBERT Test:
Accuracy: 0.8329896907216495
F1 Score: 0.8330286798650787
Precision: 0.8333052717746887
Recall: 0.8329896907216495
------
Metrics for TinyFinBERT Forex:
Accuracy: 0.47752073330423395
F1 Score: 0.45722072491516735
Precision: 0.492260544538431
Recall: 0.47752073330423395
------
Metrics for TinyFinBERT FIQA:
Accuracy: 0.5660377358490566
F1 Score: 0.5943871442980074
Precision: 0.6560374375295789
Recall: 0.5660377358490566
------
